## Data Preparation


In [130]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import os
from scipy.stats import pointbiserialr
from scipy.stats import pearsonr

#include utils directory
import sys
sys.path.append('..')

from utils.files import *
DATA_PATH = os.path.join('..', 'data')

from utils.metrics import *

### Players


In [131]:
# Players
players_teams_df = pd.read_csv(os.path.join(DATA_PATH, DATA_PLAYERS_TEAMS))
pt_df = preparePlayersTeamsDf(players_teams_df)

new_pt_df = pd.DataFrame()
for col in ['playerID', 'year', 'tmID']:
    new_pt_df[col] = pt_df[col]

teams_df = pd.read_csv(os.path.join(DATA_PATH, DATA_TEAMS))
teams_df = prepareTeamsDf(teams_df)

getPer(new_pt_df, pt_df, teams_df) #defined in metrics.py
getEFF(new_pt_df, pt_df) #defined in metrics.py

display(new_pt_df)

# drop players with less than 40 minutes played
print(new_pt_df.shape)
new_pt_df['minutes'] = pt_df['minutes']
new_pt_df = new_pt_df[new_pt_df['minutes'] >= 40]
new_pt_df = new_pt_df.drop(columns=['minutes'])
print(new_pt_df.shape)

# get the average stats for players the previous year
merged_df = teams_df[['year', 'tmID', 'playoff', 'confID']].copy()
for index, row in merged_df.iterrows():
    merged_df.loc[index, 'per'] = new_pt_df[(new_pt_df['year'] == row['year'] - 1) & (new_pt_df['tmID'] == row['tmID'])]['PER'].mean()
    merged_df.loc[index, 'eff'] = new_pt_df[(new_pt_df['year'] == row['year'] - 1) & (new_pt_df['tmID'] == row['tmID'])]['EFF'].mean()

# replace missing per with average
#merged_df['per'] = merged_df['per'].fillna(merged_df['per'].mean())
#merged_df['eff'] = merged_df['eff'].fillna(merged_df['eff'].mean())
merged_df['per'] = merged_df['per'].fillna(merged_df['per'].quantile(0.20))
merged_df['eff'] = merged_df['eff'].fillna(merged_df['eff'].quantile(0.20))

,playerID,year,tmID,PER,EFF
0,abrossv01w,2,MIN,24.671154,0.379433
1,abrossv01w,3,MIN,17.144159,0.277019
2,abrossv01w,4,MIN,23.437776,0.368177
3,abrossv01w,5,MIN,11.492671,0.304348
4,abrossv01w,6,MIN,18.800674,0.332046
...,...,...,...,...,...
1871,zakalok01w,3,PHO,-0.412979,-0.027027
1872,zarafr01w,6,SEA,7.313349,0.243736
1873,zellosh01w,10,DET,31.329987,0.429778
1874,zirkozu01w,4,WAS,0.771033,0.333333


(1876, 5)
(1690, 5)


In [132]:
players_df = pd.read_csv(os.path.join(DATA_PATH, DATA_PLAYERS))



### Awards

In [133]:
coaches_df = pd.read_csv(os.path.join(DATA_PATH, DATA_COACHES))
awards_df = pd.read_csv(f'{DATA_PATH}/{DATA_AWARDS}')

# Remove irrelevant awards
awards_df = awards_df.drop(awards_df[awards_df['award'].str.startswith('Kim Perrot')].index)

# Get the amount of awards in a team
for index, row in awards_df.iterrows():
    # add to awards_df
    tmId = players_teams_df.loc[(players_teams_df['playerID'] == row['playerID']) & (players_teams_df['year'] == row['year'])]['tmID']
    try:
        awards_df.loc[index, 'teamID'] = tmId.iloc[0] 
    except IndexError:
        try:
            tmId = coaches_df.loc[(coaches_df['coachID'] == row['playerID']) & (coaches_df['year'] == row['year'])]['tmID']
            #print(f'No team found for player {row["playerID"]} in year {row["year"]}')
            awards_df.loc[index, 'teamID'] = tmId.iloc[0]
        except:
            print(f'No team found for player {row["playerID"]} in year {row["year"]}')
            awards_df.loc[index, 'teamID'] = 'None'

teams_df_temp = teams_df.copy()
teams_df_temp['yearPrev'] = teams_df_temp['year'] - 1
for index, row in teams_df_temp.iterrows():
    teams_df.loc[index, 'awardCount'] = len(awards_df.loc[(awards_df['teamID'] == row['tmID']) & (awards_df['year'] == row['yearPrev'])].index)

teams_df[['awardCount']].describe()
merged_df['awardCount'] = teams_df['awardCount']

No team found for player coopecy01w in year 7
No team found for player boltoru01w in year 7
No team found for player weathte01w in year 7


### Teams

In [134]:
# get team conversion rate

teams_df['yearPrev'] = teams_df['year'] - 1
for index, row in teams_df.iterrows():
    try:
        prevYear = teams_df.loc[(teams_df['year'] == row['yearPrev']) & (teams_df['tmID'] == row['tmID'])]
        prevPlayers = players_teams_df.loc[(players_teams_df['tmID'] == row['tmID']) & (players_teams_df['year'] == row['yearPrev'])]['playerID']
        if len(prevPlayers) == 0:
            continue

        count = 0 
        for player in prevPlayers:
            if player not in players_teams_df.loc[(players_teams_df['tmID'] == row['tmID']) & (players_teams_df['year'] == row['year'])]['playerID'].values:
                count += 1

        teams_df.loc[index, 'teamChange'] = count / len(prevPlayers)
    except IndexError:
        pass

merged_df['teamChange'] = teams_df['teamChange']

# replace missing with avg
merged_df['teamChange'] = merged_df['teamChange'].fillna(merged_df['teamChange'].mean())

In [135]:
players_df = pd.read_csv(os.path.join(DATA_PATH, DATA_PLAYERS))
#get average eff per college
getEFF(players_teams_df, players_teams_df) #defined in metrics.py
college_df = pd.DataFrame(columns=['college', 'eff', 'count'])
for index, row in players_teams_df.iterrows():
    college = players_df.loc[players_df['bioID'] == row['playerID']]['college'].iloc[0]
    if college_df.loc[college_df['college'] == college].empty:
        # add row
        college_df.loc[len(college_df)] = [college, 0, 0]
    college_df.loc[college_df['college'] == college, 'eff'] += row['EFF']
    college_df.loc[college_df['college'] == college, 'count'] += 1

college_df['eff'] = college_df['eff'] / college_df['count']

for index, row in players_teams_df.iterrows():
    try:
        college = players_df.loc[players_df['bioID'] == row['playerID']]['college'].iloc[0]
        # if rookie
        if row['year'] == players_teams_df.loc[players_teams_df['playerID'] == row['playerID']]['year'].min():
            players_teams_df.loc[index, 'collegeEff'] = college_df.loc[college_df['college'] == college]['eff'].iloc[0]
    except IndexError:
        pass

print(players_teams_df[['collegeEff']].head())
print(players_teams_df.loc[(players_teams_df['year'] == 10) & (players_teams_df['playerID'] == 'mccouan01w')]['collegeEff'])

#add sum of eff 
for index, row in teams_df.iterrows():
    teams_df.loc[index, 'effSum'] = players_teams_df.loc[(players_teams_df['year'] == row['year']) & (players_teams_df['tmID'] == row['tmID'])]['collegeEff'].mean()
#merged_df['effSum'] = teams_df['effSum']
merged_df.head()


   collegeEff
0    0.358163
1         NaN
2         NaN
3         NaN
4         NaN
1023    0.563265
Name: collegeEff, dtype: float64


,year,tmID,playoff,confID,per,eff,awardCount,teamChange
0,9,ATL,N,EA,13.166803,0.295042,0.0,0.453509
1,10,ATL,Y,EA,11.398591,0.331618,0.0,0.642857
2,1,CHA,N,EA,13.166803,0.295042,0.0,0.453509
3,2,CHA,Y,EA,12.237540,0.306456,0.0,0.615385
4,3,CHA,Y,EA,17.755260,0.312389,0.0,0.166667


### Coach Ratings

To measure the coach efficiency, we are gonna use a simple win-loss ratio:

\begin{aligned}
& \text{Coach Rating} = \frac{\text{Wins}}{\text{Wins} + \text{Losses}}
\end{aligned}


In [136]:
coaches_df = pd.read_csv(os.path.join(DATA_PATH, DATA_COACHES))
coaches_df = prepareCoachesDf(coaches_df)

coaches_temp = coaches_df[['coachID', 'year', 'tmID']]
merged_df = pd.merge(merged_df, coaches_temp, on=['year', 'tmID'], how='left')
coaches_df['year'] = coaches_df['year'] + 1
coaches_df.drop(columns=['tmID'], inplace=True)
merged_df = pd.merge(merged_df, coaches_df, on=['year', 'coachID'], how='left')
merged_df['coachWLRatio'] = merged_df['coachWLRatio'].fillna(merged_df['coachWLRatio'].mean())
merged_df.drop(columns=['coachID'], inplace=True)

In [137]:
merged_df['playoff'] = merged_df['playoff'].eq('Y').mul(1)
merged_df.drop(columns=['tmID'], inplace=True)
merged_df.dropna(axis=0, inplace=True)
merged_df.head()

,year,playoff,confID,per,eff,awardCount,teamChange,coachWLRatio
0,9,0,EA,13.166803,0.295042,0.0,0.453509,1.351403
1,10,1,EA,11.398591,0.331618,0.0,0.642857,0.133333
2,1,0,EA,13.166803,0.295042,0.0,0.453509,1.351403
3,2,1,EA,12.237540,0.306456,0.0,0.615385,0.391304
4,3,1,EA,17.755260,0.312389,0.0,0.166667,1.222222


In [138]:
# Save the result to a new CSV file
merged_df.to_csv(os.path.join(DATA_PATH, DATA_MERGED), index=False)

### References

<a id="ref1"></a> [1] Maroun, E. (2012, March 7). Understanding advanced statistics: player efficiency rating. Hardwood Paroxysm. https://web.archive.org/web/20170910105350/https://hardwoodparoxysm.com/2012/03/07/understanding-advanced-statistics-player-efficiency-rating/

<a id="ref2"></a> [2] Calculating PER | Basketball-Reference.com. (n.d.). Basketball-Reference.com. https://www.basketball-reference.com/about/per.html